# Twitter

In this notebook we are going to show all the used for the analysis. Also we are going to show all the graphics asociated with the data obtaines.


## Introduction

We downloaded data from twitter with various methods, which did not provide the same amount of data, so we standarized to the minimal amount which it could be useful. Those fields are: 
* Date when it was created at.
* ID of the tweet, which is unique.
* The text of the tweet.
* The user, which contains the user screen name (username) and the user ID.

After defining that the period of downloading data was finished, the amount of data gathered was: **6.542.718** unique tweets. Which is a decent amount of data to analyse.

## Analysis

1) The first step is to filter the data in different files that will be used for all the analysis. The filter will be keywords (in the meantime), the keywords are the platforms name (i.e. Nintendo, Playstation, Xbox), these keywords are searched in the text and the username. If there is no defined preference (not zero preference) in which platform the record was pointing to, it is inserted in both files.

The ideal method to filter the data would be create a database of keywords asociated with each platform, so in every record when searching it could be calculated the probability of that text (according to all the words) to which platform it goes. This would require a model with N-Bayes, but like most predictive models it requires training, which we do not have at this moment.

**Note**: Also from here on we called each file as follows:
* project_tweets01.data -> Nintendo
* project_tweets02.data -> Playstation
* project_tweets03.data -> Xbox
* project_tweets04.data -> Else (Which is everything else that did not fit in the other categories)

In [ ]:
import time
import json

if __name__ == '__main__':
    time_start = time.clock()
    nintendo_platform_file = []
    playstation_platform_file = []
    xbox_platform_file = []
    else_platform_file = []
    _file = open("project_tweets.data", 'r')
    for line in _file.readlines():
        data = json.loads(line)
        try:
            raw_text = data['text'].strip().upper()
            raw_username = data['user']['screen_name'].strip().upper()
        except:
            # No idea why would happen.
            print "WTF!"
            raw_text = ""
            raw_username = ""
        nintendo_mentions = raw_text.count("NINTENDO")
        playstation_mentions = raw_text.count("PLAYSTATION")
        xbox_mentions = raw_text.count("XBOX")
        nintendo_mentions += 1 if raw_username.find("NINTENDO") != -1 else 0
        playstation_mentions += 1 if raw_username.find("PLAYSTATION") != -1 else 0
        xbox_mentions += 1 if raw_username.find("XBOX") != -1 else 0
        if nintendo_mentions > playstation_mentions and nintendo_mentions > xbox_mentions:
            nintendo_platform_file.append(line)
        elif playstation_mentions > nintendo_mentions and playstation_mentions > xbox_mentions:
            playstation_platform_file.append(line)
        elif xbox_mentions > playstation_mentions and xbox_mentions > nintendo_mentions:
            xbox_platform_file.append(line)
        elif nintendo_mentions == playstation_mentions and nintendo_mentions == xbox_mentions and nintendo_mentions > 0:
            nintendo_platform_file.append(line)
            playstation_platform_file.append(line)
            xbox_platform_file.append(line)
        elif nintendo_mentions == playstation_mentions and nintendo_mentions != xbox_mentions:
            nintendo_platform_file.append(line)
            playstation_platform_file.append(line)
        elif nintendo_mentions == xbox_mentions and nintendo_mentions != playstation_mentions:
            nintendo_platform_file.append(line)
            xbox_platform_file.append(line)
        elif playstation_mentions == xbox_mentions and playstation_mentions != nintendo_mentions:
            playstation_platform_file.append(line)
            xbox_platform_file.append(line)
        else:
            else_platform_file.append(line)
    filenames = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for filename in filenames:
        # Clean files.
        open(filename, 'w').close()
        # Write files in append mode.
        with open(filename, 'a') as _file:
            if filename == "project_tweets01.data":
                for line in nintendo_platform_file:
                    _file.write(line)
            elif filename == "project_tweets02.data":
                for line in playstation_platform_file:
                    _file.write(line)
            elif filename == "project_tweets03.data":
                for line in xbox_platform_file:
                    _file.write(line)
            else:
                for line in else_platform_file:
                    _file.write(line)
    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


2) The first analysis to do with this data that is now filtered by platforms, is to count the amount of records per platform. This is done with the next script.

In [ ]:
import time
import sys

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("records_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data", "project_tweets.data"]
    for _file in files:
        # Write to file in append mode.
        _fo = open("records_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        _data = open(_file, 'r')
        lines = _data.readlines()
        records = len(lines)
        print records, '\n'

    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**

project_tweets01.data

1317667 

project_tweets02.data

2337881 

project_tweets03.data

2260157 

project_tweets04.data

735410 

project_tweets.data

6542718 

Time taken to completion of the metric: 37.615491 in processor time

**Analysis**

From this we can see that in the "else" category we have **11.24%** of all the data, which is not a small amount. But considering that our filter for the platforms is kind of brute force is all right.

3) The next analysis to do is to count the amount of unique users per platform. This is done with the next script.

In [ ]:
import time
import sys

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("user_amount_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['user_amount.py', option1, option2, _file]
        # Write to file in append mode.
        _fo = open("user_amount_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('user_amount.py')
        print "\n"

    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import time
import itertools
import sys

class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        yield [record['user']['screen_name'], 1]

    def reducer(self, key, values):
        yield [key, 1]

    def mapper2(self, key, values):
        yield ['amount_users', values]

    def reducer2(self, key, values):
        yield [key, sum(values)]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.reducer),
                MRStep(mapper=self.mapper2, reducer=self.reducer2)]


if __name__ == '__main__':
    #time_start = time.clock()
    MRWordFrequencyCount().run()
    #time_end = time.clock()
    #print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**

project_tweets01.data

"amount_users"	426450


project_tweets02.data

"amount_users"	574308


project_tweets03.data

"amount_users"	727273


project_tweets04.data

"amount_users"	265827


Time taken to completion of the metric: 426.030981 in processor time


**Analysis**

From this we can see that in the Xbox platform there are more unique users than in all the other platforms by a not small percentage, it almost duplicates Nintendo unique users. What is interesting, is using the analysis from before we can see that Xbox had less records than Playstation, but seeing this there is a whooping 150k (estimated) more unique users in Xbox, which could lead us that Playstation content creation is more for each unique user or that there is a tiny amount of users that produce all the content for this platform.

4) The next analysis is to calculate the Top 10 users that generate the most content in each platform. This is done with the next script.

In [ ]:
import time
import sys

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("top_users_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['top_users.py', option1, option2, _file]
        # Write to file in append mode.
        _fo = open("top_users_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('top_users.py')
        print "\n"

    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import time
import itertools
import operator
import sys

class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        yield [record['user']['screen_name'], 1]

    def reducer(self, key, values):
        yield ["top_user", (sum(values), key)]

    def reducer2(self, key, values):
        user_ids = []
        user_tweets = []
        for value in values:
            user_ids.append(value[1])
            user_tweets.append(value[0])
        user = {}
        for i in xrange(0, len(user_ids)):
            user[user_ids[i]] = user_tweets[i]
        top_users = sorted(user.items(), key=lambda x: (x[1], operator.itemgetter(0)), reverse=True)
        for user in top_users[0:10]:
            #print user[0], user[1]
            yield [user[0], user[1]]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.reducer),
                MRStep(reducer=self.reducer2)]


if __name__ == '__main__':
    #time_start = time.clock()
    MRWordFrequencyCount().run()
    #time_end = time.clock()
    #print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**

project_tweets01.data
* "savetimeandmoey"	11391
* "AuctionPorn"	8449
* "AmazonBay4u"	8352
* "retrodeals"	7435
* "Nintendo_Legend"	6895
* "retrodealsUK"	6744
* "LastChanceGamer"	5904
* "GameUP247"	5580
* "RetroNuss"	5441
* "Nintendoe3E3"	5438


project_tweets02.data
* "Cammie_Whybrew"	13616
* "AskPlayStation"	13282
* "eBayShopperNews"	11578
* "VideoGamesMall"	10274
* "savetimeandmoey"	8632
* "collinschristof"	5382
* "topnewskoeln"	5297
* "Gamifive"	4829
* "Xbox_360_Gamez"	4080
* "pressebank"	3799


project_tweets03.data
* "Xbox_360_Gamez"	34137
* "XboxSupport"	14421
* "VideoGamesMall"	11835
* "Xbox_One_Reddit"	10215
* "xboxgamersdeals"	9224
* "GameUP247"	9156
* "KingsleyNewz"	9139
* "bullzyy"	7576
* "savetimeandmoey"	7176
* "giveawayxfab"	7028


project_tweets04.data
* "savetimeandmoey"	15483
* "VideoGames_Up"	13804
* "giveawaygigatop"	8878
* "tw100_1"	6538
* "videogames_pt"	6357
* "videogames_fr"	5993
* "giveawayxfab"	5886
* "DMGG_Videogames"	5295
* "ShoppeWorld"	4824
* "VideoGames_TV"	3709


Time taken to completion of the metric: 335.930534 in processor time


**Analysis**

From this we can see that there are some users in more than one platform, and that they are one of the top 10 users that generate contents, which means that they are probably spammers, bots or someone really good at making records for almost all the platforms.

5) The nexts analysis of the data is by each hour of a day in every week (the data was gathered).

5.1) We now calculate the Top 20 words. That is how many times a single word was used in each record for each platform. This is done with the next script.

In [ ]:
import time
import sys
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
import json
from datetime import datetime
from dateutil.parser import parse
import operator

# The summary is for each hour per week. (i.e. 20 times hello on week 1, 31 times bye on week 2, ...)

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("count_words_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        punctuation = []
        open(_file[:-5] + "_words.data", 'w').close()
        _output = open(_file[:-5] + "_words.data", 'a')
        _output_filename = _output.name
        for i in xrange(0, len(string.punctuation)):
            punctuation.append(string.punctuation[i])
        stop_words = stopwords.words() + punctuation
        _data = open(_file, 'r')
        for line in _data.readlines():
            data = json.loads(line)
            date = parse(data['created_at'])
            week = date.isocalendar()[1]
            hour = date.hour
            _words = [i for i in word_tokenize(data['text'].lower()) if i not in stop_words]
            for word in _words:
                record = {'word': word, 'week': week, 'hour': hour}
                _output.write(json.dumps(record, sort_keys=True) + "\n")
        _data.close()
        _output.close()
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['words_amount.py', option1, option2, _output_filename]
        # Write to file in append mode.
        _fo = open("count_words_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('words_amount.py')
        print "\n"

    time_end = time.clock()
    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)

In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import time
import itertools
import sys
from datetime import datetime
from dateutil.parser import parse
import operator

class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        hour = record['hour']
        week = record['week']
        word = record['word']
        yield [(week, hour, word), 1]

    def reducer(self, key, values):
        yield [(key[0], key[1]), (sum(values), key[2])]

    def reducer2(self, key, values):
        counts = []
        words = []
        for value in values:
            words.append(value[1])
            counts.append(value[0])
        pair_word_count = {}
        for i in xrange(0, len(words)):
            pair_word_count[words[i]] = counts[i]
        top_words = sorted(pair_word_count.items(), key=lambda x: (x[1], operator.itemgetter(0)), reverse=True)
        for word in top_words[0:20]:
            #print user[0], user[1]
            yield [(key[0], key[1], word[0]), word[1]]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.reducer), MRStep(reducer=self.reducer2)]


if __name__ == '__main__':
    #time_start = time.clock()
    MRWordFrequencyCount().run()
    #time_end = time.clock()
    #print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**




**Analysis**

From this we can see which hours in which week. The Top 20 most used words in all of that period records by platform.



** Graphic **

5.2) We now calculate how many records have a sentiment of "positiveness", "negativeness" or "neutralness" for each platform with the next script.

In [ ]:
import time
import sys
import sentiment_analyzer as sa

# The summary is for each hour per week. (i.e. 20 positives tweets on week 1, 31 negative tweets on week 2, ...)

if __name__ == '__main__':
    time_start = time.clock()
    sid = sa.init_classifier()
    # Clean File.
    open("sentiment_analysis_by_platform_summary.txt", 'w').close()
    files = ["project_tweets01.data", "project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        _output_filename = sa.classify_file(_file, sid)
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['sentiment_amount.py', option1, option2, _output_filename]
        # Write to file in append mode.
        _fo = open("sentiment_analysis_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('sentiment_amount.py')
        print "\n"

    time_end = time.clock()

    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


In [ ]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import sys


def init_classifier():
    n_instances = 100
    subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
    obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]

    train_subj_docs = subj_docs[:80]
    test_subj_docs = subj_docs[80:100]
    train_obj_docs = obj_docs[:80]
    test_obj_docs = obj_docs[80:100]
    training_docs = train_subj_docs + train_obj_docs
    testing_docs = test_subj_docs + test_obj_docs

    sentim_analyzer = SentimentAnalyzer()
    all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])
    unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)
    sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)
    training_set = sentim_analyzer.apply_features(training_docs)
    test_set = sentim_analyzer.apply_features(testing_docs)
    trainer = NaiveBayesClassifier.train
    classifier = sentim_analyzer.train(trainer, training_set)
    sid = SentimentIntensityAnalyzer()
    return sid


def classify_file(_file, sid):
    _data = open(_file, 'r')
    open(_file[:-5] + "_classified.data", 'w').close()
    _output = open(_file[:-5] + "_classified.data", 'a')
    for line in _data.readlines():
        record = json.loads(line)
        ss = sid.polarity_scores(record['text'].encode('utf-8'))
        if ss['compound'] < 0:
            classification = "neg"
        elif ss['compound'] == 0:
            classification = "neu"
        else:
            classification = "pos"
        try:
            record.update({'classification': classification})
            _output.write(json.dumps(record, sort_keys=True) + "\n")
        except:
            # Should not happen.
            print "WTF!"
    _data.close()
    _output.close()
    return _output.name

if __name__ == '__main__':
    pass


In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import time
import itertools
import sys
from datetime import datetime
from dateutil.parser import parse

class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        date = parse(record['created_at'])
        week = date.isocalendar()[1]
        hour = date.hour
        yield [(week, hour, record['classification']), 1]

    def reducer(self, key, values):
        yield [key, sum(values)]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.reducer)]


if __name__ == '__main__':
    #time_start = time.clock()
    MRWordFrequencyCount().run()
    #time_end = time.clock()
    #print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


**Results**




**Analysis**

From this we can see which hours in which week had more positive or negative records by platform.


**Graphic**

In [ ]:
import matplotlib.pyplot as plt
archivo = open("sentiment_analysis_by_platform_summary.txt")
archivo.readline()
pos = []
neg = []
neu = []
chars_to_remove = [',', '[', ']', '"']
for i in archivo:
	line = i.translate(None, ''.join(chars_to_remove))
	line = line.split()
	if line[2] == "pos":
		pos.append([int(line[0]), int(line[1]), int(line[3])])
	elif line[2] == "neg":
		neg.append([int(line[0]), int(line[1]), int(line[3])])
	else:
		neu.append([int(line[0]), int(line[1]), int(line[3])])

x_neg = []
x_neg_name = []
y_neg = []
count = 0
for i in sorted(neg):
	x_neg.append(count)
	x_neg_name.append(str(i[0])+ " " + str(i[1]))
	y_neg.append(i[2])
	count += 1

plt.xticks(x_neg, x_neg_name)
neg_plt = plt.plot(x_neg, y_neg, 'r',label='negatives')

x_neu = []
x_neu_name = []
y_neu = []
count = 0
for i in sorted(neu):
	x_neu.append(count)
	x_neu_name.append(str(i[0])+ " " + str(i[1]))
	y_neu.append(i[2])
	count += 1

plt.xticks(x_neu, x_neu_name)
neu_plt = plt.plot(x_neu, y_neu, 'b',label='neutral')


x_pos = []
x_pos_name = []
y_pos = []
count = 0
for i in sorted(pos):
	x_pos.append(count)
	x_pos_name.append(str(i[0])+ " " + str(i[1]))
	y_pos.append(i[2])
	count += 1

plt.xticks(x_pos, x_pos_name)
pos_plt = plt.plot(x_pos, y_pos, 'g', label='positive')
plt.legend(loc=2)
plt.show()

5.3) We now calculate how many records are from the different languages for each platform with the next script.

In [ ]:
import time
import sys
import language_detect as ld

# The summary is for each hour per week. (i.e. 20 spanish tweets on week 1, 31 english tweets on week 2, ...)

# Important Note: This is just an estimation of the language based on the stopwords, so if no stopwords are found the language
#                   that will produce is garbage, but is not the fault of the model. Also if there are some stopwords of both
#                   languages it will say is the one with most stopwords.

if __name__ == '__main__':
    time_start = time.clock()
    # Clean File.
    open("languages_amount_by_platform_summary.txt", 'w').close()
    files = ["project_tweets02.data", "project_tweets03.data", "project_tweets04.data"]
    for _file in files:
        _output_filename = ld.set_language_for_each_tweet(_file)
        # parameters for mrjob.
        # To run your job in multiple subprocesses with a few Hadoop features simulated, use -r local.
        option1 = "" #""-r"
        option2 = "" #""local"
        sys.argv = ['count_language.py', option1, option2, _output_filename]
        # Write to file in append mode.
        _fo = open("languages_amount_by_platform_summary.txt", 'a')
        sys.stdout = _fo
        print _file
        execfile('count_language.py')
        print "\n"

    time_end = time.clock()
    print "Time taken to completion of the metric: {0} in processor time".format(time_end - time_start)


In [ ]:
import sys
import json
reload(sys)
sys.setdefaultencoding('utf8')
try:
    from nltk import wordpunct_tokenize
    from nltk.corpus import stopwords
except ImportError:
    print '[!] You need to install nltk (http://nltk.org/index.html)'



#----------------------------------------------------------------------
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    
    @param text: Text whose language want to be detected
    @type text: str
    
    @return: Dictionary with languages and unique stopwords seen in analyzed text
    @rtype: dict
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    # Compute per language included in nltk number of unique stopwords appearing in analyzed text
    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios


#----------------------------------------------------------------------
def detect_language(text):
    """
    Calculate probability of given text to be written in several languages and
    return the highest scored.
    
    It uses a stopwords based approach, counting how many unique stopwords
    are seen in analyzed text.
    
    @param text: Text whose language want to be detected
    @type text: str
    
    @return: Most scored language guessed
    @rtype: str
    """

    ratios = _calculate_languages_ratios(text)

    most_rated_language = max(ratios, key=ratios.get)

    return most_rated_language


def set_language_for_each_tweet(_file):
    _data = open(_file, 'r')
    open(_file[:-5] + "_language.data", 'w').close()
    _output = open(_file[:-5] + "_language.data", 'a')
    for line in _data.readlines():
        record = json.loads(line)
        language = detect_language(record['text'].encode('utf-8'))
        try:
            record.update({'language': language})
            _output.write(json.dumps(record, sort_keys=True) + "\n")
        except:
            # Should not happen.
            print "WTF!"
    _data.close()
    _output.close()
    return _output.name



In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
from datetime import datetime
import itertools


class MRWordFrequencyCount(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, _, record):
        yield [record['language'], 1]

    def reducer(self, key, values):
        yield [key, sum(values)]

    def steps(self):
        return [MRStep(mapper=self.mapper, reducer=self.max_reducer)]

if __name__ == '__main__':
    MRWordFrequencyCount.run()


**Results**



**Analysis**

From this we can see which hours in which week had more records made in the different languages by platform.


** Graphic **